In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [3]:
import random

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    
    # Realistic Distance matrix (in km)
    data['distance_matrix'] = [
        [0, 4, 12, 9, 7],  # Warehouse to other locations
        [4, 0, 6, 5, 3],   # Point A to other locations
        [12, 6, 0, 8, 5],  # Point B to other locations
        [9, 5, 8, 0, 4],   # Point C to other locations
        [7, 3, 5, 4, 0]    # Point D to other locations
    ]
    
    # Vehicle details (weight capacity in kg)
    data['vehicle_capacities'] = [20, 25]  # Two vehicles, with different capacities
    
    # Delivery times in minutes (randomly assigned for more realism)
    data['delivery_times'] = [10, 15, 12, 8, 11]  # Time taken to deliver to each point
    
    # Number of vehicles and depot location (Warehouse)
    data['num_vehicles'] = 2  # Two vehicles
    data['depot'] = 0  # Warehouse
    
    # Randomized parcel sizes (in kg) for deliveries
    data['parcel_sizes'] = [5, 8, 3, 6, 4]  # Simulate parcel sizes for each delivery point
    
    return data

def main():
    """Solves the VRP with realistic constraints."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint (vehicles can't exceed their load capacity)
    def demand_callback(from_index):
        """Returns the demand (parcel size) at the node."""
        from_node = manager.IndexToNode(from_index)
        return data['parcel_sizes'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data['vehicle_capacities'],  # vehicle capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Time constraint (time to complete deliveries)
    def time_callback(from_index):
        """Returns the delivery time at the node."""
        from_node = manager.IndexToNode(from_index)
        return data['delivery_times'][from_node]

    time_callback_index = routing.RegisterUnaryTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        0,  # no slack
        100,  # max time allowed (this can be adjusted)
        True,  # start cumul to zero
        'Time')

    # Setting the first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue()))
    total_distance = 0
    total_time = 0
    
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_time = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            route_time += routing.GetDimensionOrDie('Time').CumulVar(index)
        
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {} km\n'.format(route_distance)
        plan_output += 'Time taken: {} minutes\n'.format(route_time)
        print(plan_output)
        total_distance += route_distance
        total_time += route_time
    
    print('Total distance of all routes: {} km'.format(total_distance))
    print('Total time of all routes: {} minutes'.format(total_time))

if __name__ == '__main__':
    main()


Objective: 37
Route for vehicle 0:
 0 -> 3 -> 2 -> 4 -> 0
Distance of the route: 29 km
Time taken: (((Time3(0..100) + Time2(0..100)) + Time4(0..100)) + Time6(0..100)) minutes

Route for vehicle 1:
 0 -> 1 -> 0
Distance of the route: 8 km
Time taken: (Time1(0..100) + Time7(0..100)) minutes

Total distance of all routes: 37 km
Total time of all routes: ((((Time3(0..100) + Time2(0..100)) + Time4(0..100)) + Time6(0..100)) + (Time1(0..100) + Time7(0..100))) minutes
